In [30]:
import seaborn as sns
import polars as pl
from pathlib import Path
import json

In [39]:
mem_dir = Path('/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/scripts/training_mem/memray')
mem_json = list(mem_dir.glob('*.json'))
res = {'n_seq': [], 'length': [], 'peak_mem': []}
for path in mem_json:
    res['n_seq'].append(int(path.stem.split('_')[2].split('=')[1]))
    res['length'].append(int(path.stem.split('_')[3].split('=')[1]))
    with open(path) as io:
        res['peak_mem'].append(json.load(io)['total_bytes_allocated'])

In [40]:
res_df = pl.DataFrame(res)
res_df

n_seq,length,peak_mem
i64,i64,i64
10000000,100,11666523275
1000000,10000,16073743896
1000000,100000,72456228791
100000000,1000,87214532383
1000000,5000,10093414728
10000000,5000,14834586389
1000,1000,3157719266
1000000,1000,5296719919
10000000,10000,18258473591
